In [1]:
from pyssemble import Pyssemble

In [ ]:
# initialize Pyssemble at an instruction Memory Address which is optional. 
# This imports all the necessary register variables and assembly instructions in current environment.
Pyssemble.initiate(400)

In [2]:
# Simple Assembly code to load an immediate value to register a1.
# The program will get loaded into memory at the address 400.
Li(a1, 5)
Ret()

In [ ]:
# start executing program at address 400. It will stop at error or Ret
Pyssemble.execute(400)

In [3]:
# way 1 to see result
a1.value

5

In [4]:
# Note x11 is same as a1. Way 2 to see result
x11.value

5

In [5]:
# Way 3 and most pythonic way to see the result
print(a1)

5


In [6]:
# Debugger allows us to have more control over the overall internals.
from pyssemble import Debugger

In [7]:
# Way 4 to see result. List all 32 registers. Way to track chenges in registers.
Debugger.show_all_regs(col_width=5)

0    0    0    0    0    0    0    0    0    0    0    5    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    
x0   x1   x2   x3   x4   x5   x6   x7   x8   x9   x10  x11  x12  x13  x14  x15  x16  x17  x18  x19  x20  x21  x22  x23  x24  x25  x26  x27  x28  x29  x30  x31  


In [ ]:
# initialise debugging instructions into environment
Debugger.initiate()

In [8]:
# way 5 to see result. Also PrintR is an added Assembly Instruction so can be loaded and executed within the program.
PrintR(a1).execute()

5
